# Tweets about the top companies 
# Dor Azulay & Netanel Kluzner 
-------------------------------------------

In [ ]:
!pip install yfinance
!pip install TwitterAPI
!pip install timedelta

In [ ]:
 # imports & initialisations
import csv
import math
import pandas as pd
import numpy as np
import datetime
import timedelta
import time
import io
import plotly.graph_objects as go
from dateutil import parser
from sklearn import linear_model
from textblob import TextBlob # sentiment score
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
users_path = '/content/drive/MyDrive/Alon/לימודים/Final Project/Data/Self Collected/users_with_eng_v5(with_replies).csv'

tweets_path = "/content/drive/MyDrive/Alon/לימודים/Final Project/Data/From the Web/TweetsAboutTopCompanies/Tweet.csv/Tweet.csv"
tweets_2019_path = "/content/drive/MyDrive/Alon/לימודים/Final Project/Data/From the Web/TweetsAboutTopCompanies/tweets_2019.csv"
filteredTweetsWithFlair_path = "/content/drive/MyDrive/Alon/לימודים/Final Project/Data/From the Web/TweetsAboutTopCompanies/tweets_2019_flairSentiment.csv"


In [ ]:
# auths
def connectToTwitterAPI1():
  import tweepy # tweeter API
  from datetime import datetime, timedelta

  consumer_key = 'mymakG2knztQ2GYaTNaRGTIOi'
  consumer_secret = 'lGXmXu9K7DQftUjvVempNg1vGjS362zbKo7p12yaa5RrBelIlj'
  access_token = '561299890-kjoCtIBYvSHeIVfhYEbHfNXHAqVklnMze2Wce1JT'
  access_token_secret = 'U1fR9nx24H1Eo3dvlLYO9LhDxwYWWZ5x2JEltgA4xUy6o'

  auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_token, access_token_secret)

  return tweepy.API(auth)

def connectToTwitterAPI2():
  from TwitterAPI import TwitterAPI, TwitterOAuth, TwitterRequestError, TwitterConnectionError, TwitterPager
  consumer_key = 'mymakG2knztQ2GYaTNaRGTIOi'
  consumer_secret = 'lGXmXu9K7DQftUjvVempNg1vGjS362zbKo7p12yaa5RrBelIlj'
  access_token = '561299890-kjoCtIBYvSHeIVfhYEbHfNXHAqVklnMze2Wce1JT'
  access_token_secret = 'U1fR9nx24H1Eo3dvlLYO9LhDxwYWWZ5x2JEltgA4xUy6o'
  return TwitterAPI (consumer_key, consumer_secret, access_token,
  access_token_secret, api_version='2')

In [ ]:
# get the currect dataframes from drive in "paths" section
users_df = pd.read_csv(users_path)
tweets_2019 = pd.read_csv(tweets_2019_path)
users_df

,name,screen_name,id,id_str,description,followers_count,favourites_count,listed_count,friends_count,statuses_count,created_at,verified,eng_total_retweets,eng_total_likes,eng_total_replies,eng_tweets_length,eng_score
0,PowerTriggerTrade,Sandro_power,3346438133,3346438133,"Big Picture Leader Trading Stocks Options and Health Expert Leader Demonstrate Powerful Options Plans many Documented on my YouTube Channel, WITNESS THE POWER",3742,2367,70,14,10088,Fri Jun 26 11:31:16 +0000 2015,False,1287.0,1426.0,825.0,100.0,0.0
1,Michaël van de Poppe,CryptoMichNL,146008010,146008010,"CEO & Founder Eight, @eight_global || Full Time Trader at Beursplein 5, Amsterdam || BSc Economics || #Bitcoin, #Crypto & Monetary Systems || TA / PA trader",539091,46077,5057,738,64574,Thu May 20 11:34:14 +0000 2010,False,7876.0,91957.0,17164.0,100.0,0.0
2,Johns Charts,johnscharts,3410575617,3410575617,Tweeting with friends and sharing charts. My tweets are my thoughts and for ideas only. Not investment advice. Trade your plan. I will never solicit thru DM's!,127699,1037,2515,48,105567,Sun Aug 09 16:10:04 +0000 2015,False,0.0,0.0,0.0,0.0,0.0
3,mcm-ct.com,mcm_ct,3162911248,3162911248,"Markets centric datamodeling, analytics/technology, finance impact on society. Opinions solely for info/educational purposes + @mcm_ct_markets #ticktools feed",21955,55099,564,4065,250150,Mon Apr 13 08:06:30 +0000 2015,False,175.0,535.0,78.0,74.0,0.0
4,◇.A.C.X.◇,ACXtrades,2929743297,2929743297,There is no passion to be found playing small - in settling for a life that is less than the one you are capable of living.,17487,12739,456,324,13584,Thu Dec 18 16:18:24 +0000 2014,False,83.0,1118.0,247.0,100.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2447,David Holt,IDrawCharts,958032765710143489,958032765710143489,Opinion not for sale.\n\nFounder @JungleDefi and @Round_Table_TG,43628,149388,617,693,52294,Mon Jan 29 17:43:05 +0000 2018,False,130.0,5166.0,566.0,100.0,0.0
2448,Ian Dyer,IanDyerGuru,1083461136719269891,1083461136719269891,Senior Investment Analyst and Editor at Banyan Hill Publishing.,5110,621,38,107,834,Thu Jan 10 20:30:39 +0000 2019,False,142.0,1765.0,224.0,100.0,0.0
2449,@coach23,coach23life,3008460077,3008460077,Life is about the means. Load can be heavy but supporting each other we can lift it up. Believe in electric transport & solar energy generation and storage,193,10656,0,769,10363,Sat Jan 31 20:53:34 +0000 2015,False,19.0,155.0,17.0,100.0,0.0
2450,Elon Promises,ElonPromises,1189930459146473474,1189930459146473474,"Parody account with sh*t Elon said, newly arranged. Non-commercial. Based on—but not affiliated with—@tedstein’s wonderful https://t.co/7f03G8sfdH. $tslaq",462,44,7,0,2330,Thu Oct 31 15:42:17 +0000 2019,False,47.0,304.0,12.0,100.0,0.0


In [ ]:
def getEngScore(users_df, include_replies=True, include_followers=True):
  df = users_df
  for i in range(len(df)):
    rts = df['eng_total_retweets'][i]
    likes = df['eng_total_likes'][i]
    replies = df['eng_total_replies'][i]
    total_tweets = df['eng_tweets_length'][i]
    followers = df['followers_count'][i]
    if include_followers:
      df['eng_score'][i] = ((rts+likes+replies)/total_tweets)/followers if include_replies else ((rts+likes)/total_tweets)/followers
    else:
      df['eng_score'][i] = ((rts+likes+replies)/total_tweets) if include_replies else ((rts+likes)/total_tweets)
  return df


users_df=getEngScore(users_df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning:

divide by zero encountered in double_scalars



In [ ]:
users_df = pd.read_csv(users_path)

In [ ]:
amateur_threshold = 2000
amateur_df = users_df[users_df['followers_count']<=amateur_threshold]
users_df = users_df[users_df['followers_count']>amateur_threshold].reset_index(drop=True)
influencers_df = users_df[users_df['verified'] == True]
influencers_df = influencers_df.reset_index(drop=True)
stocks_keywords =['stock', 'exchange', 'market', 'ceo', 'manag', 'invest', 'crypto', 'trad', 'founder', 'author', 'reporter', 'analysis', 'analyst', 'research', 'infla', 'bond', 'earnings', 'dividend', 'econom', 'financ']

influencers_df['stocks_related'] = [0 for i in range(len(influencers_df))]

for i in range(len(influencers_df)):
  desc = influencers_df['description'][i].lower()
  for word in stocks_keywords:
    if word in desc:
      influencers_df['stocks_related'][i] = 1

influencers_df = influencers_df.reset_index(drop=True)


stocks_related_df = influencers_df[influencers_df['stocks_related'] == 1]
influencers_df = influencers_df[influencers_df['stocks_related'] == 0]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
print(len(amateur_df), len(influencers_df), len(stocks_related_df))


1008 153 229


In [ ]:

#!pip install nltk
import nltk
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer


nltk.download('stopwords')
stopwords_lst = stopwords.words('english')
", ".join(stopwords_lst)

# unified_str = ''
# for desc in influencers_df['description']:
#   unified_str += desc


#remove the stopwords from the texts
#remove the stopwords from the texts
def clean_stopwords(text):
  return " ".join([word for word in str(text).split() if word not in stopwords_lst])

influencers_df['description'] =   influencers_df['description'].apply(lambda text : clean_stopwords(text))

punctuations = string.punctuation


#removing punctuations
punctuations = string.punctuation
def clean_punctuation(text):
  translator = str.maketrans('', '' ,punctuations)
  return text.translate(translator)
influencers_df['description']= influencers_df['description'].apply(lambda x: clean_punctuation(x))


#cleaning repeating characters
def remove_repeats(text):
  return  re.sub(r'(.)\1+', r'\1', text)
influencers_df['description']= influencers_df['description'].apply(lambda x: remove_repeats(x))

#clean email data
def clean_email(data):
  return re.sub('@[^\s]+', ' ', data)
influencers_df['description']= influencers_df['description'].apply(lambda x: clean_email(x))


#clean urls
def clean_urls(data):
  return re.sub('((www\.[^\s]+) | (https?://[^\s]))', ' ', data)
influencers_df['description']= influencers_df['description'].apply(lambda x: clean_urls(x))

#clean numeric numbers
def clean_numbers(data):
  return re.sub('[0-9]+', ' ', data)
influencers_df['description']= influencers_df['description'].apply(lambda x: clean_numbers(x))

#get tokenization of tweet text
tokenizer = RegexpTokenizer(r'\w+')
influencers_df['description']= influencers_df['description'].apply(tokenizer.tokenize)


#apply stemming and lemmatizer
#stemming
import nltk
st = nltk.PorterStemmer()
def stemming(data):
  text = [st.stem(word) for word in data]
  return data
influencers_df['description'] = influencers_df['description'].apply(lambda x: stemming(x))

#lemmatizer
lm = nltk.WordNetLemmatizer()
nltk.download('wordnet')

def lemmatizer(data):
  text = [lm.lemmatize(word) for word in data]
  return data
influencers_df['description'] = influencers_df['description'].apply(lambda x: lemmatizer(x))


influencers_df



In [ ]:
words_dict ={}
for words in influencers_df['description']:
  for word in words:
    words_dict[word] =0

for words in influencers_df['description']:
  for word in words:
    words_dict[word] += 1

sort_by_value = dict(sorted(words_dict.items(), key=lambda item: item[1], reverse=True))

sort_by_value



In [ ]:
influencers_df

In [ ]:
stocks_keywords =['stock', 'exchange', 'market', 'ceo', 'manag', 'invest', 'crypto', 'trad', 'founder', 'author', 'reporter', 'analysis', 'analyst', 'research', 'infla', 'bond', 'earnings', 'dividend', 'econom', 'financ']

influencers_df['stocks_related'] = [0 for i in range(len(influencers_df))]

for i in range(len(influencers_df)):
  desc = influencers_df['description'][i].lower()
  for word in stocks_keywords:
    if word in desc:
      influencers_df['stocks_related'][i] = 1

influencers_df = influencers_df.reset_index(drop=True)


stocks_related_df = influencers_df[influencers_df['stocks_related'] == 1]
influencers_df = influencers_df[influencers_df['stocks_related'] == 0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
print(len(amateur_df), len(influencers_df), len(stocks_related_df))
influenceres_df
stocks_related_df

109 153 229


,name,screen_name,id,id_str,description,followers_count,favourites_count,listed_count,friends_count,statuses_count,created_at,verified,eng_total_retweets,eng_total_likes,eng_total_replies,eng_tweets_length,eng_score,stocks_related
1,Stocktwits,Stocktwits,14886375,14886375,The global voice of finance and investing.,938796,48665,12275,14744,50702,Fri May 23 20:53:49 +0000 2008,True,881.0,4899.0,1854.0,100.0,0.0,1
2,Scott Wapner,ScottWapnerCNBC,263197731,263197731,"Host, CNBC's Halftime Report. Author of “When the Wolves Bite,” Dad, Lesser half of the Han-Wapner dynasty. Lover of food and DC sports. Fan of the truth.",206753,9250,1454,732,20416,Wed Mar 09 15:50:30 +0000 2011,True,132.0,3492.0,1416.0,100.0,0.0,1
4,Abigail Doolittle,TheChartress,398105054,398105054,"Markets for @BloombergTV, Host of Charting Futures & Smart Charts, love quotes, spirit & all things UES. Often smiling, always grateful.",20965,14932,528,1089,12220,Tue Oct 25 15:48:38 +0000 2011,True,296.0,3492.0,228.0,100.0,0.0,1
5,HowardLindzon,howardlindzon,7517052,7517052,"GP @socialleverage, CEO $SLAC the SPAC, co-founder @stocktwits founded Wallstrip (acquired by CBS), seed @robinhoodapp @onrallyrd @koyfincharts @etoro @alpacahq",276131,80884,5169,1046,48454,Mon Jul 16 21:30:12 +0000 2007,True,204.0,4676.0,790.0,100.0,0.0,1
6,CNBC's Fast Money,CNBCFastMoney,16732770,16732770,"Fast Money with @MelissaLeeCNBC airs weeknights at 5P ET, only on @CNBC. America's post-market show.",428721,4677,5486,1682,54149,Tue Oct 14 03:16:30 +0000 2008,True,1437.0,5579.0,1116.0,100.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,WallStreetBΞts,wallstreetbets,588569122,588569122,Jaime Rogozinski - Founder WallStreetBets\n\nTG https://t.co/7m0dIVlGfW▫️\nDiscord https://t.co/mrCXBrQfaw\nOs https://t.co/ou5vuA39EL,581284,1380,2449,386,6155,Wed May 23 20:42:12 +0000 2012,True,5502.0,32172.0,3477.0,100.0,0.0,1
375,Kenny Polcari,KennyPolcari,300929831,300929831,"Managing Partner Kace Capital Advisors, Market Analyst Fox Biz, CNN, CNBC Key Note Speaker, @HeadstrongProj. Tweets are my own & not reflective of employer.",14582,6929,298,598,24950,Wed May 18 15:56:10 +0000 2011,True,182.0,1497.0,325.0,100.0,0.0,1
376,Kevin Fox is hopeful 🦊💉🩹 ❤️,kfury,785,785,"Designer/Founder. Gmail, Google Calendar, Google Reader, FriendFeed, Electric Imp and other stuff. Now I make tweets and art. he/him",21471,55346,743,550,89531,Fri Jul 14 07:42:04 +0000 2006,True,62.0,620.0,140.0,100.0,0.0,1
377,S3 Partners,S3Partners,2964381029,2964381029,"FinTech Company. Providing reporting, data and analytics solutions for better outcomes in investment processes, risk management, and counterparty relationships.",42013,2899,331,619,4359,Tue Jan 06 15:50:03 +0000 2015,True,490.0,3765.0,454.0,73.0,0.0,1


In [ ]:
def saveToCSV(df, file_name):
  #export to csv
  compression_opts = dict(method='zip',
                          archive_name=file_name + '.csv')
  df.to_csv(file_name + '.zip', index=False,
            compression=compression_opts)

saveToCSV(influencers_df, 'influencers')
saveToCSV(amateur_df,'amatuers')
saveToCSV(stocks_related_df,'stock_related')